In [ ]:
# Routines to perform Validation of YUEGE algorithms

In [1]:
# Please copy this to any new notebook 
import os, sys, time, math
from IPython.display import display
import pandas as pd

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
from bipolo import db

# Engine is the key to executing raw sql
engine = db.engine

In [3]:
# Just to see if we're on the local server
sql = """SELECT COUNT(*) FROM business""" 
biz_count = pd.read_sql(sql, engine)
display(biz_count)   # 3550 on local

c:\Anaconda\lib\site-packages\sqlalchemy\dialects\mysql\base.py:1569: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


,COUNT(*)
0,3550


In [4]:
# Pick some Phoenix restaurants at random
sql = """SELECT name, city, id FROM business 
         WHERE city = 'Phoenix' LIMIT 10""" 
rated_rests = pd.read_sql(sql, engine)

# Convert to list and then to a tuple that can be used for SQL "IN"
rest_ids = rated_rests.loc[:, 'id'].values.tolist()   # Why do I have to slice it again to make it work?
rest_ids = [str(item) for item in rest_ids]      # Drop the dreaded unicode label
if len(rest_ids) == 1:
    rest_ids = "('"+ rest_ids[0] + "')"        # Avoid tuple trailing comma for single elements
else:
    rest_ids = tuple(rest_ids)

display(rated_rests)
display(rest_ids)


,name,city,id
0,Kabab House,Phoenix,--g-a85VwrdZJNf0R95GcQ
1,Matt's Big Breakfast,Phoenix,-050d_XIor1NpCuWkbIVaQ
2,Tea2go,Phoenix,-0aIra_B6iALlfqAriBSYA
3,Dubliner,Phoenix,-0tgMGl7D9B10YjSN2ujLA
4,Game Seven Grill,Phoenix,-0WegMt6Cy966qlDKhu6jA
5,Matt's Big Breakfast,Phoenix,-1UMR00eXtwaeh59pEiDjA
6,Taco Bell,Phoenix,-2isRNVb6PDuBagELL5EBw
7,La Fontanella Restaurant,Phoenix,-8JaNeG3etLgOVyg2gOIiA
8,The Melting Pot,Phoenix,-9eNGMp8XiygI8t8QFuFWw
9,Zen 32,Phoenix,-A9sm-E6uQxWTJ_MuyOzFw


('--g-a85VwrdZJNf0R95GcQ',
 '-050d_XIor1NpCuWkbIVaQ',
 '-0aIra_B6iALlfqAriBSYA',
 '-0tgMGl7D9B10YjSN2ujLA',
 '-0WegMt6Cy966qlDKhu6jA',
 '-1UMR00eXtwaeh59pEiDjA',
 '-2isRNVb6PDuBagELL5EBw',
 '-8JaNeG3etLgOVyg2gOIiA',
 '-9eNGMp8XiygI8t8QFuFWw',
 '-A9sm-E6uQxWTJ_MuyOzFw')

In [5]:
# Identify # reviews for each restaurant 
sql = """SELECT business_id, COUNT(user_id) FROM review 
         WHERE business_id IN {xxx} 
         GROUP BY business_id ORDER BY COUNT(user_id) DESC LIMIT 10"""
sql = """SELECT business_id, COUNT(user_id) FROM review 
         WHERE business_id IN {xxx} 
         GROUP BY business_id"""
sql = sql.format(xxx=rest_ids)
rest_review_counts = pd.read_sql(sql, engine)
display(rest_review_counts)



,business_id,COUNT(user_id)
0,--g-a85VwrdZJNf0R95GcQ,24
1,-050d_XIor1NpCuWkbIVaQ,700
2,-0aIra_B6iALlfqAriBSYA,22
3,-0tgMGl7D9B10YjSN2ujLA,112
4,-0WegMt6Cy966qlDKhu6jA,74
5,-1UMR00eXtwaeh59pEiDjA,279
6,-2isRNVb6PDuBagELL5EBw,16
7,-8JaNeG3etLgOVyg2gOIiA,61
8,-9eNGMp8XiygI8t8QFuFWw,244
9,-A9sm-E6uQxWTJ_MuyOzFw,105


In [21]:
# Select a restaurant
rest1 = '-0tgMGl7D9B10YjSN2ujLA'
# '--g-a85VwrdZJNf0R95GcQ'
# '-050d_XIor1NpCuWkbIVaQ'
# '-0aIra_B6iALlfqAriBSYA'
# '-0tgMGl7D9B10YjSN2ujLA'
# '-0WegMt6Cy966qlDKhu6jA'
# '-1UMR00eXtwaeh59pEiDjA'
# '-2isRNVb6PDuBagELL5EBw'
# '-8JaNeG3etLgOVyg2gOIiA'
# '-9eNGMp8XiygI8t8QFuFWw'
# '-A9sm-E6uQxWTJ_MuyOzFw'


In [22]:
# Get user_ids that have reviewed the selected restaurant 
sql = """SELECT user_id FROM review WHERE business_id = '{xxx}'"""
sql = sql.format(xxx=rest1)
reviewers = pd.read_sql(sql, engine)

# Convert to list and then to a tuple that can be used for SQL "IN"
user_ids = reviewers.loc[:, 'user_id'].values.tolist()       # Why do I have to slice it again to make it work?
user_ids = [str(item) for item in user_ids]      # Drop the dreaded unicode label
if len(user_ids) == 1:
    user_ids = "('"+ user_ids[0] + "')"        # Avoid tuple trailing comma for single elements
else:
    user_ids = tuple(user_ids)
display(user_ids)    

('YSvMlcF2k6KtUCedgfsYwg',
 'kc-rnN-ndnFTdHG4TfIgeQ',
 'Pm23QTyzjbwXfKHp8lEzhA',
 'tDPCBslkz5ULjuyR77YNfg',
 'eAJ3Ip0vh-zuk_BmcmchYA',
 'R4eLJKwG9yQ44vfwaUHS3g',
 'HP9dSSNzz0EbCM-oIAfu8g',
 'fwJr8mZu9cWTplq3Cm4dqA',
 'F1d0aQcDvhwn9EfXC6vgMw',
 'drDGFLwDwuVBnjBQmFyXVg',
 'PWeEk_eX0nsqGRkq9nmZfQ',
 'wAB16R8AAR_RVPMEjjxvrQ',
 'ViFZS37P8JMMQo6KGrK0sA',
 'SxSUs5K8ny2MtnVo_aJ-Qg',
 'kpZWl_uXA262ji6wt6phpg',
 'C1u8tkqX3X7nPB3tjKCqXQ',
 '0etjqbarS9quNVP-F7lHbg',
 'H3iIzvfpTGavCv1hQwsgZw',
 'AjrMEX47GaTeFw36vfVS-A',
 'pgbMIGVJuX_9CgO045brVQ',
 'U3FhLnEGCYqlGZ5uoEqfXg',
 'qPl66x3bYyLONUsYQ_TJNA',
 'rWz-qRVQPtOs2pH5GSA71g',
 'ePfhvRM-deQgMduESHRWDg',
 '5dSxCFW9nBJhi-yt_3Px8g',
 '11K8CM8LZ0vwiLELSsE_EA',
 '1dbluwhDAd8EoBJOMtiSag',
 'm-bOe0jjQB8-cUw7CZ4J4Q',
 'OK9t7c_Ew8iB8AwkM4r0Jg',
 'jPy5x1WstUOtKI9x51UQ-g',
 '1R_GlcglxZaNkH2ya26kDQ',
 'REoakEnDCB6pJ4WQpu04Wg',
 'uNu-d-yuprzEEpGQj28Pxw',
 'lfwVWEuy44kKxhoBl2H6pg',
 'kJsMFBdQ_Do67u3lpvaujA',
 '3HhotfEkRci6A0mCt0EDhQ',
 'RParK1O1EhA5dDJQw05JSg',
 

In [23]:
# Find other restaurants that these users have reviewed, counting how many per restaurant  
start_time = time.time()
sql = """SELECT business_id, COUNT(user_id) AS count, user_id 
         FROM review WHERE user_id IN {xxx} 
         GROUP BY business_id ORDER BY count DESC LIMIT 10"""
sql = sql.format(xxx=user_ids)
common_rests = pd.read_sql(sql, engine)
print("--- %s seconds ---" % (time.time() - start_time)) 
display(common_rests)

# Convert to list and then to a tuple that can be used for SQL "IN"
common_rest_ids = common_rests.loc[:, 'business_id'].values.tolist()       # Why do I have to slice it again to make it work?
common_rest_ids = [str(item) for item in common_rest_ids]      # Drop the dreaded unicode label
if len(common_rest_ids) == 1:
    common_rest_ids = "('"+ common_rest_ids[0] + "')"        # Avoid tuple trailing comma for single elements
else:
    common_rest_ids = tuple(common_rest_ids)
display(common_rest_ids)

--- 1.4430000782 seconds ---


,business_id,count,user_id
0,-0tgMGl7D9B10YjSN2ujLA,112,-ajqjNKLQUtthACttISgvw
1,7m1Oa1VYV98UUuo_6i0EZg,14,1VIuDm-PsXw7Yfd98UdYpQ
2,p9v9Sl2g_ibaajWvO0NjkQ,11,-ajqjNKLQUtthACttISgvw
3,cHdJXLlKNWixBXpDwEGb_A,11,92e_B3VDY5TqxVYsKuNzKg
4,042IHd5KjHiMuBtGtugO_g,11,-ajqjNKLQUtthACttISgvw
5,9a3DrZvpYxVs3k_qwlCNSw,10,92e_B3VDY5TqxVYsKuNzKg
6,_WtxQbDK7B-ExGdeG-2j6Q,10,121ruQUFk6Bl2V1wBwa31A
7,tcgmfbWugkHI3FRrjlH6lQ,9,1dbluwhDAd8EoBJOMtiSag
8,zbrFk-4ejesAJD8EwcdHxg,9,-ajqjNKLQUtthACttISgvw
9,E4JyAzB5_2quptwtemyhYA,9,2VhVWgmjlJ0V9S9V0S3buA


('-0tgMGl7D9B10YjSN2ujLA',
 '7m1Oa1VYV98UUuo_6i0EZg',
 'p9v9Sl2g_ibaajWvO0NjkQ',
 'cHdJXLlKNWixBXpDwEGb_A',
 '042IHd5KjHiMuBtGtugO_g',
 '9a3DrZvpYxVs3k_qwlCNSw',
 '_WtxQbDK7B-ExGdeG-2j6Q',
 'tcgmfbWugkHI3FRrjlH6lQ',
 'zbrFk-4ejesAJD8EwcdHxg',
 'E4JyAzB5_2quptwtemyhYA')

In [24]:
# Get the users that have reviewed multiple restaurants  
start_time = time.time()
sql = """SELECT user_id FROM review WHERE user_id IN {xxx} AND business_id IN {yyy} GROUP BY user_id"""
sql = sql.format(xxx=user_ids, yyy=common_rest_ids)
common_users = pd.read_sql(sql, engine)
print("--- %s seconds ---" % (time.time() - start_time)) 
display(common_users)

# Convert to list and then to a tuple that can be used for SQL "IN"
common_user_ids = common_users.loc[:, 'user_id'].values.tolist()       # Why do I have to slice it again to make it work?
common_user_ids = [str(item) for item in common_user_ids]      # Drop the dreaded unicode label
if len(common_user_ids) == 1:
    common_user_ids = "('"+ common_user_ids[0] + "')"        # Avoid tuple trailing comma for single elements
else:
    common_user_ids = tuple(common_user_ids)
display(common_user_ids)

--- 1.43700003624 seconds ---


,user_id
0,-ajqjNKLQUtthACttISgvw
1,0etjqbarS9quNVP-F7lHbg
2,11K8CM8LZ0vwiLELSsE_EA
3,121ruQUFk6Bl2V1wBwa31A
4,1dbluwhDAd8EoBJOMtiSag
5,1dQiuUvvK8427TBmqj_4Vg
6,1R_GlcglxZaNkH2ya26kDQ
7,1VIuDm-PsXw7Yfd98UdYpQ
8,2VhVWgmjlJ0V9S9V0S3buA
9,2yqjFChucO1DDv-bFnZoBg


('-ajqjNKLQUtthACttISgvw',
 '0etjqbarS9quNVP-F7lHbg',
 '11K8CM8LZ0vwiLELSsE_EA',
 '121ruQUFk6Bl2V1wBwa31A',
 '1dbluwhDAd8EoBJOMtiSag',
 '1dQiuUvvK8427TBmqj_4Vg',
 '1R_GlcglxZaNkH2ya26kDQ',
 '1VIuDm-PsXw7Yfd98UdYpQ',
 '2VhVWgmjlJ0V9S9V0S3buA',
 '2yqjFChucO1DDv-bFnZoBg',
 '3HhotfEkRci6A0mCt0EDhQ',
 '5dSxCFW9nBJhi-yt_3Px8g',
 '5xJuhHjwVfzPuQQHl4G1Sw',
 '6U-RmQAW1BB_QdO_6emA0w',
 '6VI_AXIdglQPPnQnhyp7ng',
 '92e_B3VDY5TqxVYsKuNzKg',
 'AjrMEX47GaTeFw36vfVS-A',
 'b8LFVfj8Gu3FWJB38b-EDQ',
 'bCTdxXoKOuzIEsov9zxOMQ',
 'bhWmsFBPk3bPeMCzucWmUg',
 'C1u8tkqX3X7nPB3tjKCqXQ',
 'cRQ3UCd1NKi2jxHZpvZlTQ',
 'CwLEKnnODT2zSpqzsLojFw',
 'cXR7PnThzRyDU6SoDcIyGA',
 'd1JSdP3ch8mlVS0RPnj6qw',
 'DJ6KIZ5h_clUqcXvgnJjwA',
 'drDGFLwDwuVBnjBQmFyXVg',
 'e8lMbWebomEsTgr-5CGjMQ',
 'eAJ3Ip0vh-zuk_BmcmchYA',
 'ePfhvRM-deQgMduESHRWDg',
 'F1d0aQcDvhwn9EfXC6vgMw',
 'FBTKVybXaUuRZEbaZeohbw',
 'ffPY_bHX8vLebHu8LBEqfg',
 'fjAOk0_RMr83mJBkWWjEQQ',
 'fuU0Q1_uhS8zXqNKfmdEWg',
 'fwJr8mZu9cWTplq3Cm4dqA',
 'F_VibKB1NLgl-sHMVeWZ_Q',
 

In [25]:
# Get the ratings of these users across the different restaurants  

sql = "DROP TABLE IF EXISTS ratings"
engine.execute(sql)

start_time = time.time()
sql = """CREATE TABLE IF NOT EXISTS ratings AS (
         SELECT b.name AS restaurant, b.id AS rest_id, r.user_id, r.stars AS rating 
         FROM review AS r
         JOIN business AS b ON r.business_id = b.id 
         WHERE r.user_id IN {xxx} AND r.business_id IN {yyy}
         ORDER BY r.user_id)"""
sql = sql.format(xxx=common_user_ids, yyy=common_rest_ids)
engine.execute(sql)

sql = "SELECT * FROM ratings"
ratings = pd.read_sql(sql, engine)
print("--- %s seconds ---" % (time.time() - start_time)) 
display(ratings)

--- 4.24899983406 seconds ---


,restaurant,rest_id,user_id,rating
0,Yard House,p9v9Sl2g_ibaajWvO0NjkQ,-ajqjNKLQUtthACttISgvw,4
1,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw,4
2,Valle Luna,zbrFk-4ejesAJD8EwcdHxg,-ajqjNKLQUtthACttISgvw,4
3,OHSO Brewery - Paradise Valley,042IHd5KjHiMuBtGtugO_g,-ajqjNKLQUtthACttISgvw,3
4,Dubliner,-0tgMGl7D9B10YjSN2ujLA,0etjqbarS9quNVP-F7lHbg,1
5,Dubliner,-0tgMGl7D9B10YjSN2ujLA,11K8CM8LZ0vwiLELSsE_EA,5
6,Dubliner,-0tgMGl7D9B10YjSN2ujLA,121ruQUFk6Bl2V1wBwa31A,3
7,Rokerij,_WtxQbDK7B-ExGdeG-2j6Q,121ruQUFk6Bl2V1wBwa31A,4
8,Rokerij,_WtxQbDK7B-ExGdeG-2j6Q,1dbluwhDAd8EoBJOMtiSag,4
9,Dubliner,-0tgMGl7D9B10YjSN2ujLA,1dbluwhDAd8EoBJOMtiSag,3


In [26]:
# Create a list of user pairs that have reviewed the same restaurant and calculate the jaccard similarity of their ratings
# Note: user1 < user2 prevents user1-user1 pairs AND eliminates duplicate user2-user1 pairs 
# This ordering of user1-user2 alphabetically will let us find matching user pairs later 
start_time = time.time()
sql = "DROP TABLE IF EXISTS user_pairs1"
engine.execute(sql)
print("--- %s seconds ---" % (time.time() - start_time)) 

start_time = time.time()
sql = """CREATE TABLE IF NOT EXISTS user_pairs1 AS (
         SELECT r1.restaurant, r1.rest_id, 
         CONCAT(r1.user_id, '+++', r2.user_id) AS user_pair,
         LEAST(r1.rating, r2.rating) / GREATEST(r1.rating, r2.rating) AS rating_match
         FROM ratings AS r1
         LEFT JOIN ratings AS r2 ON r1.rest_id = r2.rest_id
         WHERE r1.user_id < r2.user_id
         ORDER BY r1.user_id ASC)"""
engine.execute(sql)
print("--- %s seconds ---" % (time.time() - start_time)) 

sql = """SELECT * FROM user_pairs1"""
user_pairs = pd.read_sql(sql, engine)
display(user_pairs)

--- 0.018000125885 seconds ---
--- 0.255000114441 seconds ---


,restaurant,rest_id,user_pair,rating_match
0,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++2VhVWgmjlJ0V9S9V0S3buA,0.7500
1,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++OHK6Ic_73DqIryviNe88eg,0.7500
2,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++QO39oQBDvskG8uJ4RiS26g,1.0000
3,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++1R_GlcglxZaNkH2ya26kDQ,0.7500
4,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++q2B124odCQB5SWTrdbXPZw,0.7500
5,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++REoakEnDCB6pJ4WQpu04Wg,1.0000
6,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++cXR7PnThzRyDU6SoDcIyGA,0.5000
7,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++q1lTTRLIhCfsultUG1zwUA,0.2500
8,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++o0md1gJyn4KmKvEPghEGPg,0.7500
9,Dubliner,-0tgMGl7D9B10YjSN2ujLA,-ajqjNKLQUtthACttISgvw+++H0A1fKEM4AdYurSS560WfA,0.7500


In [27]:
# Find user pairs that have rated more than one restaurant in common
# Same trick of < to avoid duplicate restaurant pairs
start_time = time.time()
sql = """SELECT up1.user_pair, up1.restaurant AS restaurant1, up1.rating_match AS match1,
         up2.restaurant AS restaurant2, up2.rating_match AS match2
         FROM user_pairs1 AS up1
         JOIN user_pairs1 AS up2 ON up1.user_pair = up2.user_pair 
         WHERE up1.rest_id < up2.rest_id
         ORDER BY up1.user_pair ASC"""
common_reviews = pd.read_sql(sql, engine)
print("--- %s seconds ---" % (time.time() - start_time)) 
display(common_reviews)

--- 4.65199995041 seconds ---


,user_pair,restaurant1,match1,restaurant2,match2
0,-ajqjNKLQUtthACttISgvw+++1VIuDm-PsXw7Yfd98UdYpQ,Dubliner,0.8000,OHSO Brewery - Paradise Valley,1.0000
1,-ajqjNKLQUtthACttISgvw+++2VhVWgmjlJ0V9S9V0S3buA,Dubliner,0.7500,OHSO Brewery - Paradise Valley,1.0000
2,-ajqjNKLQUtthACttISgvw+++92e_B3VDY5TqxVYsKuNzKg,OHSO Brewery - Paradise Valley,0.7500,Yard House,1.0000
3,-ajqjNKLQUtthACttISgvw+++92e_B3VDY5TqxVYsKuNzKg,Dubliner,1.0000,Yard House,1.0000
4,-ajqjNKLQUtthACttISgvw+++92e_B3VDY5TqxVYsKuNzKg,Dubliner,1.0000,OHSO Brewery - Paradise Valley,0.7500
5,-ajqjNKLQUtthACttISgvw+++AjrMEX47GaTeFw36vfVS-A,Dubliner,0.8000,Valle Luna,1.0000
6,-ajqjNKLQUtthACttISgvw+++CwLEKnnODT2zSpqzsLojFw,Dubliner,0.8000,OHSO Brewery - Paradise Valley,0.7500
7,-ajqjNKLQUtthACttISgvw+++e8lMbWebomEsTgr-5CGjMQ,OHSO Brewery - Paradise Valley,0.6000,Valle Luna,1.0000
8,-ajqjNKLQUtthACttISgvw+++e8lMbWebomEsTgr-5CGjMQ,Dubliner,0.5000,OHSO Brewery - Paradise Valley,0.6000
9,-ajqjNKLQUtthACttISgvw+++e8lMbWebomEsTgr-5CGjMQ,Dubliner,0.5000,Valle Luna,1.0000


In [28]:
# Save to file for R data analysis
#common_reviews.to_csv('kabab_house.csv', sep='\t', encoding='utf-8')
common_reviews.to_csv('dubliner.csv', encoding='utf-8')